In [61]:
from main import from_xy

In [12]:
import geopandas as gpd
countries = gpd.read_file('data/countries.geojson')
print(countries.head())

         ADMIN ISO_A3                                           geometry
0        Aruba    ABW  POLYGON ((-69.99694 12.57758, -69.93639 12.531...
1  Afghanistan    AFG  POLYGON ((71.04980 38.40866, 71.05714 38.40903...
2       Angola    AGO  MULTIPOLYGON (((11.73752 -16.69258, 11.73851 -...
3     Anguilla    AIA  MULTIPOLYGON (((-63.03767 18.21296, -63.09952 ...
4      Albania    ALB  POLYGON ((19.74777 42.57890, 19.74601 42.57993...


In [13]:
countries[countries.ADMIN == 'India']

,ADMIN,ISO_A3,geometry
104,India,IND,"MULTIPOLYGON (((93.85532 7.21418, 93.86500 7.2..."


In [14]:
nb_polygons, nb_vertices = 0, 0

for i in range(0, len(countries)):
    polygons = countries.iloc[i, 2]
    try:
        x, y = polygons.exterior.coords.xy
        nb_polygons += 1
        nb_vertices += len(x)
    except AttributeError:
        polygons = list(polygons)
        for p in polygons:
            x, y = p.exterior.coords.xy
            nb_polygons += 1
            nb_vertices += len(x)

In [15]:
nb_polygons, nb_vertices # A lot of vertices!

(4252, 548038)

Now the objective is to build a dictionary having every country as keys, and a list of its polygons as values. We also need another dictionary that keeps track of the countries center [IF NEEDED].   
As the dataset is quite large, we also need to build three separate dataset : small, medium, and large.

In [81]:
from data import Country
countries_list = []
for i in range(0, len(countries)):
    name = countries.iloc[i, 0]
    polygons = countries.iloc[i, 2]
    try:
        x, y = polygons.exterior.coords.xy
        poly = [from_xy(x, y)]
    except:
        poly = []
        for p in polygons:
            x, y = p.exterior.coords.xy
            poly.append(from_xy(x, y))

    countries_list.append(Country(poly, name))
    countries_list[len(countries_list) - 1].batch()

In [82]:
for c in countries_list:
    print(c)

Aruba: 1 polygon.s ; big: 52 ; medium: 26 ; small: 6
Afghanistan: 1 polygon.s ; big: 3066 ; medium: 1533 ; small: 307
Angola: 3 polygon.s ; big: 3290 ; medium: 1645 ; small: 330
Anguilla: 2 polygon.s ; big: 56 ; medium: 28 ; small: 6
Albania: 1 polygon.s ; big: 1114 ; medium: 557 ; small: 112
Aland: 16 polygon.s ; big: 944 ; medium: 472 ; small: 99
Andorra: 1 polygon.s ; big: 102 ; medium: 51 ; small: 11
United Arab Emirates: 10 polygon.s ; big: 1570 ; medium: 785 ; small: 161
Argentina: 8 polygon.s ; big: 9402 ; medium: 4701 ; small: 943
Armenia: 2 polygon.s ; big: 834 ; medium: 417 ; small: 84
American Samoa: 5 polygon.s ; big: 124 ; medium: 62 ; small: 14
Antarctica: 179 polygon.s ; big: 47672 ; medium: 23836 ; small: 4837
Ashmore and Cartier Islands: 1 polygon.s ; big: 10 ; medium: 5 ; small: 1
French Southern and Antarctic Lands: 18 polygon.s ; big: 1066 ; medium: 533 ; small: 114
Antigua and Barbuda: 2 polygon.s ; big: 216 ; medium: 108 ; small: 22
Australia: 94 polygon.s ; big: 

In [ ]:
with open('countries_list', 'wb') as f:
    pickle.dump(countries_list, f)

In [132]:
def pickle_load(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

cl = pickle_load('countries_list')

In [126]:
def flatten(array):
    while not check(array):
        for a in array:
            if type(a) is list:
                array += a
                array.remove(a)
    return array

def check(array):
    for a in array:
        if type(a) is not float:
            return False
    return True

In [156]:
v = [c.polygons_big for c in cl]

In [157]:
a = []
for i in v:
    for j in i:
        a += j

In [159]:
low_x, high_x, low_y, high_y = 0, 0, 0, 0
for i in range(len(a)):
    if i % 2 == 0:
        low_x = a[i] if a[i] < low_x else low_x
        high_x = a[i] if a[i] > high_x else high_x
    else:
        low_y = a[i] if a[i] < low_y else low_y
        high_y = a[i] if a[i] > high_y else high_y

low_x, high_x, low_y, high_y

(-180.0, 180.0000000000002, -83.63410065300012, 90.0)